# Extract sources from catalogs  in  OGA

 work with Weakly_2024_07
- use jupyter kernel LSST
- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2024/02/18
- update : 2024/02/24


         DatasetType('isolated_star_cat', {instrument, skymap, tract}, DataFrame),
         DatasetType('isolated_star_sources', {instrument, skymap, tract}, DataFrame),
         DatasetType('isolated_star_sources_atlas_refcat2_20220201_match', {skymap, tract}, DataFrame),


In [1]:
! eups list -s | grep LOCAL

analysis_tools        LOCAL:/home/d/dagoret/repos/repos_w_2024_07/analysis_tools 	setup
atmospec              LOCAL:/home/d/dagoret/repos/repos_w_2024_07/atmospec 	setup
eups                  LOCAL:/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-8.0.0/eups 	setup
spectractor           LOCAL:/home/d/dagoret/repos/repos_w_2024_07/Spectractor 	setup


In [2]:
! eups list -s lsst_distrib

   g4213664e8e+34187316c4 	current w_2024_07 setup


In [3]:
import lsst.daf.butler as dafButler
#import lsst.summit.utils.butlerUtils as butlerUtils

In [4]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm


import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits
from astropy.table import Table
import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [5]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time


In [6]:
# LSST Display
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [7]:
transform = AsinhStretch() + PercentileInterval(99.)

In [8]:
pd.options.display.max_columns = None
#pd.options.display.max_rows = None



## Configuration

In [9]:
DATE = 202402
#filesource_out="finalized_src_table_202402.csv"
#filetractpatch_out = 'Latiss_tractspatchs_202312.pickle'

In [10]:
#repo =  "/sdf/group/rubin/repo/main"
repo = "/sdf/group/rubin/repo/oga/"
butler = dafButler.Butler(repo)
registry = butler.registry

In [11]:
#file_out="sourceTable_202312.csv"

## Give the collection

In [12]:
#my_collection = "LATISS/runs/AUXTEL_DRP_IMAGING_2023-11A-10A-09AB-08ABC-07AB-05AB/w_2023_46/PREOPS-4553"
my_collection = "LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240201/w_2024_05/PREOPS-4871"

## Config

# Sources

In [13]:
# src : a Source catalog
# srcMatchFull
# finalized_src_table : sape of a pandas dataframe

#'isolated_star_cat', {instrument, skymap, tract}, DataFrame),
#'isolated_star_sources', {instrument, skymap, tract}, DataFrame),
#'isolated_star_sources_atlas_refcat2_20220201_match', {skymap, tract}, DataFrame),

#data_product = "isolated_star_sources" 
data_product = "isolated_star_cat" 
#data_product = 'isolated_star_sources_atlas_refcat2_20220201_match' # Nothing
datasetRefs = butler.registry.queryDatasets(datasetType=data_product, collections=my_collection, where= "instrument='LATISS'")
filesource_out=f"{data_product}_{DATE}.csv"

In [14]:
all_df = []

for i, ref in enumerate(datasetRefs):
    print(f"========================datasetType = {data_product} ============================================")
    print("fullId.............................:",ref.dataId.full)
    print("tract..............................:",ref.dataId['tract'])
    tract = ref.dataId['tract']
    df = butler.get(ref)

    if(len(df)>0):
        print(type(df),len(df))
        df['tract'] = tract
        all_df.append(df)
    

========================datasetType = isolated_star_cat ============================================
fullId.............................: {instrument: 'LATISS', skymap: 'latiss_v1', tract: 454}
tract..............................: 454
========================datasetType = isolated_star_cat ============================================
fullId.............................: {instrument: 'LATISS', skymap: 'latiss_v1', tract: 8188}
tract..............................: 8188


/tmp/ipykernel_16230/1033383477.py:5: FutureWarning: Call to deprecated function (or staticmethod) full. (DataCoordinate.full is deprecated in favor of .mapping, and will be dropped after v27.) -- Deprecated since version v27.
  print("fullId.............................:",ref.dataId.full)
/tmp/ipykernel_16230/1033383477.py:5: FutureWarning: Call to deprecated function (or staticmethod) full. (DataCoordinate.full is deprecated in favor of .mapping, and will be dropped after v27.) -- Deprecated since version v27.
  print("fullId.............................:",ref.dataId.full)


<class 'pandas.core.frame.DataFrame'> 314
========================datasetType = isolated_star_cat ============================================
fullId.............................: {instrument: 'LATISS', skymap: 'latiss_v1', tract: 5616}
tract..............................: 5616
<class 'pandas.core.frame.DataFrame'> 476
========================datasetType = isolated_star_cat ============================================
fullId.............................: {instrument: 'LATISS', skymap: 'latiss_v1', tract: 10644}
tract..............................: 10644
<class 'pandas.core.frame.DataFrame'> 442
========================datasetType = isolated_star_cat ============================================
fullId.............................: {instrument: 'LATISS', skymap: 'latiss_v1', tract: 455}
tract..............................: 455


/tmp/ipykernel_16230/1033383477.py:5: FutureWarning: Call to deprecated function (or staticmethod) full. (DataCoordinate.full is deprecated in favor of .mapping, and will be dropped after v27.) -- Deprecated since version v27.
  print("fullId.............................:",ref.dataId.full)
/tmp/ipykernel_16230/1033383477.py:5: FutureWarning: Call to deprecated function (or staticmethod) full. (DataCoordinate.full is deprecated in favor of .mapping, and will be dropped after v27.) -- Deprecated since version v27.
  print("fullId.............................:",ref.dataId.full)
/tmp/ipykernel_16230/1033383477.py:5: FutureWarning: Call to deprecated function (or staticmethod) full. (DataCoordinate.full is deprecated in favor of .mapping, and will be dropped after v27.) -- Deprecated since version v27.
  print("fullId.............................:",ref.dataId.full)


<class 'pandas.core.frame.DataFrame'> 2093
========================datasetType = isolated_star_cat ============================================
fullId.............................: {instrument: 'LATISS', skymap: 'latiss_v1', tract: 452}
tract..............................: 452
<class 'pandas.core.frame.DataFrame'> 119
========================datasetType = isolated_star_cat ============================================
fullId.............................: {instrument: 'LATISS', skymap: 'latiss_v1', tract: 5615}
tract..............................: 5615


...

<class 'pandas.core.frame.DataFrame'> 523
========================datasetType = isolated_star_cat ============================================
fullId.............................: {instrument: 'LATISS', skymap: 'latiss_v1', tract: 380}
tract..............................: 380
========================datasetType = isolated_star_cat ============================================
fullId.............................: {instrument: 'LATISS', skymap: 'latiss_v1', tract: 3863}
tract..............................: 3863
<class 'pandas.core.frame.DataFrame'> 245
========================datasetType = isolated_star_cat ============================================
fullId.............................: {instrument: 'LATISS', skymap: 'latiss_v1', tract: 5839}
tract..............................: 5839
<class 'pandas.core.frame.DataFrame'> 18
========================datasetType = isolated_star_cat ============================================
fullId.............................: {instrument: 'LATISS', skyma

/tmp/ipykernel_16230/1033383477.py:5: FutureWarning: Call to deprecated function (or staticmethod) full. (DataCoordinate.full is deprecated in favor of .mapping, and will be dropped after v27.) -- Deprecated since version v27.
  print("fullId.............................:",ref.dataId.full)
/tmp/ipykernel_16230/1033383477.py:5: FutureWarning: Call to deprecated function (or staticmethod) full. (DataCoordinate.full is deprecated in favor of .mapping, and will be dropped after v27.) -- Deprecated since version v27.
  print("fullId.............................:",ref.dataId.full)
/tmp/ipykernel_16230/1033383477.py:5: FutureWarning: Call to deprecated function (or staticmethod) full. (DataCoordinate.full is deprecated in favor of .mapping, and will be dropped after v27.) -- Deprecated since version v27.
  print("fullId.............................:",ref.dataId.full)
/tmp/ipykernel_16230/1033383477.py:5: FutureWarning: Call to deprecated function (or staticmethod) full. (DataCoordinate.full i

<class 'pandas.core.frame.DataFrame'> 47
========================datasetType = isolated_star_cat ============================================
fullId.............................: {instrument: 'LATISS', skymap: 'latiss_v1', tract: 5634}
tract..............................: 5634
<class 'pandas.core.frame.DataFrame'> 2564
========================datasetType = isolated_star_cat ============================================
fullId.............................: {instrument: 'LATISS', skymap: 'latiss_v1', tract: 5614}
tract..............................: 5614
<class 'pandas.core.frame.DataFrame'> 447
========================datasetType = isolated_star_cat ============================================
fullId.............................: {instrument: 'LATISS', skymap: 'latiss_v1', tract: 10643}
tract..............................: 10643
<class 'pandas.core.frame.DataFrame'> 14
========================datasetType = isolated_star_cat ============================================
fullId.............

/tmp/ipykernel_16230/1033383477.py:5: FutureWarning: Call to deprecated function (or staticmethod) full. (DataCoordinate.full is deprecated in favor of .mapping, and will be dropped after v27.) -- Deprecated since version v27.
  print("fullId.............................:",ref.dataId.full)
/tmp/ipykernel_16230/1033383477.py:5: FutureWarning: Call to deprecated function (or staticmethod) full. (DataCoordinate.full is deprecated in favor of .mapping, and will be dropped after v27.) -- Deprecated since version v27.
  print("fullId.............................:",ref.dataId.full)
/tmp/ipykernel_16230/1033383477.py:5: FutureWarning: Call to deprecated function (or staticmethod) full. (DataCoordinate.full is deprecated in favor of .mapping, and will be dropped after v27.) -- Deprecated since version v27.
  print("fullId.............................:",ref.dataId.full)
/tmp/ipykernel_16230/1033383477.py:5: FutureWarning: Call to deprecated function (or staticmethod) full. (DataCoordinate.full i

<class 'pandas.core.frame.DataFrame'> 1350


In [16]:
df = pd.concat(all_df)

In [17]:
df

,isolated_star_id,ra,dec,primary_band,source_cat_index,nsource,source_cat_index_i,nsource_i,source_cat_index_z,nsource_z,source_cat_index_r,nsource_r,source_cat_index_g,nsource_g,source_cat_index_y,nsource_y,source_cat_index_u,nsource_u,tract
0,108188,253.548157,-8.750126,i,0,10,0,4,4,0,4,4,8,2,10,0,10,0,8188
1,208188,253.492415,-8.747484,i,10,9,10,4,14,0,14,3,17,2,19,0,19,0,8188
2,308188,253.548515,-8.755819,i,19,5,19,4,23,0,23,1,24,0,24,0,24,0,8188
3,408188,253.525449,-8.764117,i,24,10,24,4,28,0,28,4,32,2,34,0,34,0,8188
4,508188,253.512081,-8.804718,i,34,11,34,4,38,0,38,4,42,3,45,0,45,0,8188
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345,134603864,122.636921,-36.166699,y,152519,1,152519,0,152519,0,152519,0,152519,0,152519,1,152520,0,3864
1346,134703864,122.627452,-36.261952,y,152520,1,152520,0,152520,0,152520,0,152520,0,152520,1,152521,0,3864
1347,134803864,122.584535,-36.077190,y,152521,1,152521,0,152521,0,152521,0,152521,0,152521,1,152522,0,3864
1348,134903864,122.571455,-36.266611,y,152522,1,152522,0,152522,0,152522,0,152522,0,152522,1,152523,0,3864


In [18]:
df.to_csv(filesource_out)

In [19]:
print(df.columns.tolist())

['isolated_star_id', 'ra', 'dec', 'primary_band', 'source_cat_index', 'nsource', 'source_cat_index_i', 'nsource_i', 'source_cat_index_z', 'nsource_z', 'source_cat_index_r', 'nsource_r', 'source_cat_index_g', 'nsource_g', 'source_cat_index_y', 'nsource_y', 'source_cat_index_u', 'nsource_u', 'tract']


In [20]:
df['tract'].unique()

array([ 8188,  5616, 10644,   455,   452,  5615,  3863,  5839,   383,
         382,  5634,  5614, 10643,  3864])

In [21]:
ref.to_simple()

SerializedDatasetRef(id=UUID('1bcd85e8-3690-4f94-8728-b87d2af74806'), datasetType=SerializedDatasetType(name='isolated_star_cat', storageClass='DataFrame', dimensions=['instrument', 'skymap', 'tract'], parentStorageClass=None, isCalibration=False), dataId=SerializedDataCoordinate(dataId={'instrument': 'LATISS', 'skymap': 'latiss_v1', 'tract': 3864}, records=None), run='LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240201/w_2024_05/PREOPS-4871/20240206T181631Z', component=None)